In [16]:
from Crypto.Util.number import *
from hashlib import sha256
from gmssl import sm2, sm4


p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0x0000000000000000000000000000000000000000000000000000000000000000
b = 0x0000000000000000000000000000000000000000000000000000000000000007
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
ord_ = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

F = GF(p)
E = EllipticCurve(F, [a, b])
G = E(Gx, Gy)

def KDF(data, klen):
    return int(sha256(data.encode()).hexdigest(), 16)%(2^klen-1)


def PGP_ke(klen):
    h = E.order()//G.order()
    mask = 2^128-1
    #A: Alice long-term pk
    a = randint(1, ord_-1)
    A = a*G
    #B: Bob long-term pk
    b = randint(1, ord_-1)
    B = b*G
    #x: Alice sk
    x = randint(1, ord_-1)
    X = x*G
    #y: Bob sk
    y = randint(1, ord_-1)
    Y = y*G
    
    
    x_ = ZZ(X.xy()[0])&mask
    y_ = ZZ(Y.xy()[0])&mask
    tA = a+x_*x
    tB = b+y_*y
    
    Za = h*tA*(B+y_*Y)
    Zb = h*tB*(A+x_*X)
    
    assert Za == Zb
    
    K = KDF(str(Za.xy()[0])+str(Za.xy()[1]), klen)
    print("Key exchange:", K)
    

def PGP_enc(msg, sk, pk):
    sm2_ = sm2.CryptSM2(public_key=pk, private_key=sk)
    return sm2_.encrypt(msg)
    

def PGP_dec(ct, sk, pk):
    sm2_ = sm2.CryptSM2(public_key=pk, private_key=sk)
    return sm2_.decrypt(ct)

def PGP_sym_enc(m, key, iv):
    sm4_ = sm4.CryptSM4()
    sm4_.set_key(key, sm4.SM4_ENCRYPT)
    return sm4_.crypt_cbc(iv, m)
    
def PGP_sym_dec(m, key, iv):
    sm4_ = sm4.CryptSM4(sm4.SM4_DECRYPT)
    sm4_.set_key(key, sm4.SM4_DECRYPT)
    return sm4_.crypt_cbc(iv, m)


def test():
    iv = os.urandom(16)
    key = os.urandom(16)
    PGP_ke(128)
    m = b'hash_hash'
    sk = '00B9AB0B828FF68872F21A837FC303668428DEA11DCD1B24429D0C99E24EED83D5'
    pk = 'B9C9A6E04E9C91F7BA880429273747D7EF5DDEB0BB2FF6317EB00BEF331A83081A6994B8993F3F5D6EADDDB81872266C87C018FB4162F5AF347B483E24620207'
    c = PGP_enc(m, sk, pk)
    print("SM2 Enc :", c.hex())
    m_ = PGP_dec(c, sk, pk)
    print("SM2 Dec :", m_)
    C = PGP_sym_enc(m, key, iv)
    print("SM4 Enc :", C.hex())
    M = PGP_sym_dec(C, key, iv)
    print("SM4 Dec :", M)
    

if __name__ == '__main__':
    test()

Key exchange: 90906877669797933897379736638685872497
SM2 Enc : 5efeb4e7f5d0c3f34ceafa2776178e29ad4b7e1c847dddacfe9641fe061fb67bf3ae0d11eea40ce3a16d81f8a0a8d4d308d60d43d68ce9e2bbfa426f891664b1d1fb43b7dd232f171814773738193e625a823797004eea98686161d771d60d3c1823636bd5de9e189b
SM2 Dec : b'hash_hash'
SM4 Enc : df834ef218958e5292a561aeb0e0f5a0
SM4 Dec : b'hash_hash'
